Note: All Outputs Cleared except for Data 1 and Data 2 Evaluation cell outputs. 
The Outputs, particularly the plots were making the notebook size very large. 

Supervised Machine Learning Methods. 
Linear Regression Prediction Simple Models to predict O3 (Ozone) measures. 
Explored by performing prediction models using 2 different cleansed datasets: 
1. Without Zero or Null Values
2. With Zero Values replaced with Median Values of their respective columns

In [7]:
# Import Modules and Packages. 
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

Dataset 1 data_drop.csv

In [8]:
# Dataset1. Read in cleansed data file 'data_drop.csv' from Temporary AWS S3 Bucket. Zipped data files in the Resource folder. 
url="https://project-4-group-6-air-quality.s3.us-east-2.amazonaws.com/data_drop.csv"
air_data_df = pd.read_csv(
    url,
    sep=',',
    encoding='utf-8',
)
air_data_df.head()

,Unnamed: 0,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM
0,0,2013,3,1,0,9.0,9.0,3.0,17.0,300.0,89.0,-0.5,1024.5,-21.4,0.0,NNW,5.7
1,1,2013,3,1,1,4.0,4.0,3.0,16.0,300.0,88.0,-0.7,1025.1,-22.1,0.0,NW,3.9
2,2,2013,3,1,5,4.0,4.0,9.0,25.0,300.0,78.0,-2.4,1027.5,-21.3,0.0,NW,2.4
3,3,2013,3,1,6,5.0,5.0,10.0,29.0,400.0,67.0,-2.5,1028.2,-20.4,0.0,NW,2.2
4,4,2013,3,1,7,3.0,6.0,12.0,40.0,400.0,52.0,-1.4,1029.5,-20.4,0.0,NNW,3.0


In [9]:
# Dataset1. View the data type and column counts
air_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382168 entries, 0 to 382167
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  382168 non-null  int64  
 1   year        382168 non-null  int64  
 2   month       382168 non-null  int64  
 3   day         382168 non-null  int64  
 4   hour        382168 non-null  int64  
 5   PM2.5       382168 non-null  float64
 6   PM10        382168 non-null  float64
 7   SO2         382168 non-null  float64
 8   NO2         382168 non-null  float64
 9   CO          382168 non-null  float64
 10  O3          382168 non-null  float64
 11  TEMP        382168 non-null  float64
 12  PRES        382168 non-null  float64
 13  DEWP        382168 non-null  float64
 14  RAIN        382168 non-null  float64
 15  wd          382168 non-null  object 
 16  WSPM        382168 non-null  float64
dtypes: float64(11), int64(5), object(1)
memory usage: 49.6+ MB


In [10]:
# Dataset1. Drop the 'wd' string object, and date columns, and save to new df
air_df = air_data_df.drop(columns=["Unnamed: 0",'year', 'month', 'day','hour','wd'])
air_df

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
0,9.0,9.0,3.0,17.0,300.0,89.0,-0.5,1024.5,-21.4,0.0,5.7
1,4.0,4.0,3.0,16.0,300.0,88.0,-0.7,1025.1,-22.1,0.0,3.9
2,4.0,4.0,9.0,25.0,300.0,78.0,-2.4,1027.5,-21.3,0.0,2.4
3,5.0,5.0,10.0,29.0,400.0,67.0,-2.5,1028.2,-20.4,0.0,2.2
4,3.0,6.0,12.0,40.0,400.0,52.0,-1.4,1029.5,-20.4,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...
382163,11.0,11.0,2.0,2.0,200.0,99.0,11.7,1008.9,-13.3,0.0,1.3
382164,13.0,13.0,2.0,2.0,200.0,101.0,10.9,1009.0,-14.0,0.0,2.1
382165,9.0,14.0,2.0,2.0,200.0,102.0,9.5,1009.4,-13.0,0.0,1.5
382166,10.0,12.0,2.0,2.0,200.0,97.0,7.8,1009.6,-12.6,0.0,1.4


In [11]:
# Dataset1. Get the metrics of columns in the dataframe
air_df.describe()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
count,382168.000000,382168.000000,382168.000000,382168.000000,382168.000000,382168.000000,382168.000000,382168.000000,382168.000000,382168.000000,382168.000000
mean,79.432383,104.573837,15.634814,50.570068,1229.940563,57.376676,13.518694,1010.813471,2.417195,0.065030,1.738031
std,80.154901,91.379446,21.306103,35.062086,1157.151476,56.709013,11.425355,10.452381,13.798402,0.823901,1.241152
min,2.000000,2.000000,0.285600,2.000000,100.000000,0.214200,-19.900000,982.400000,-36.000000,0.000000,0.000000
25%,20.000000,36.000000,2.000000,23.000000,500.000000,10.495800,3.100000,1002.400000,-9.000000,0.000000,0.900000
50%,55.000000,82.000000,7.000000,43.000000,900.000000,45.000000,14.400000,1010.400000,3.000000,0.000000,1.400000
75%,111.000000,145.000000,19.000000,71.000000,1500.000000,82.000000,23.200000,1019.000000,15.100000,0.000000,2.200000
max,844.000000,999.000000,500.000000,290.000000,10000.000000,1071.000000,41.600000,1042.800000,29.100000,72.500000,13.200000


In [ ]:
# Dataset1. Create a scatter plot
air_plot = air_df.hvplot.scatter(
    x="TEMP",
    y="O3",
    title="Dataset 1 Expected Ozone Measures based on Temperature"
)
air_plot

Dataset1. Linear Regression Model to predict Ozone based on Temperature. 

In [12]:
# Dataset1. Create the X set by using the `reshape` function to format the TEMP data as a single column array.
X = air_df["TEMP"].values.reshape(-1, 1)
# Display sample data
X[:5]

array([[-0.5],
       [-0.7],
       [-2.4],
       [-2.5],
       [-1.4]])

In [13]:
# Dataset1. Create an array for the Target (dependent) variable y with O3 data
y = air_df["O3"]

Dataset1. Linear Regression Model with SciKit-learn

In [14]:
# Dataset1. Create a model with scikit-learn
model = LinearRegression()

In [15]:
# Dataset1. Fit the data into the model
model.fit(X, y)

LinearRegression()

In [16]:
# Dataset1. Display the slope
print(f"Model's slope: {model.coef_}")

Model's slope: [2.96365606]


In [17]:
# Dataset1. Display the y-intercept
print(f"Model's y-intercept: {model.intercept_}")

Model's y-intercept: 17.31191562019925


In [18]:
# Dataset1. Display the model's best fit line formula
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")

Model's formula: y = 17.31191562019925 + 2.963656055355665X


Dataset1. Plot of Best Fit Line for the Prediction Model

In [19]:
# Dataset1. Make predictions using the X set
predicted_y_values = model.predict(X)

In [20]:
# Dataset1. Create a copy of the original data
air_ozone_predicted = air_df.copy()

# Add a column with the predicted sales values
air_ozone_predicted["Ozone_predicted"] = predicted_y_values

# Display sample data
air_ozone_predicted.head()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM,Ozone_predicted
0,9.0,9.0,3.0,17.0,300.0,89.0,-0.5,1024.5,-21.4,0.0,5.7,15.830088
1,4.0,4.0,3.0,16.0,300.0,88.0,-0.7,1025.1,-22.1,0.0,3.9,15.237356
2,4.0,4.0,9.0,25.0,300.0,78.0,-2.4,1027.5,-21.3,0.0,2.4,10.199141
3,5.0,5.0,10.0,29.0,400.0,67.0,-2.5,1028.2,-20.4,0.0,2.2,9.902775
4,3.0,6.0,12.0,40.0,400.0,52.0,-1.4,1029.5,-20.4,0.0,3.0,13.162797


In [ ]:
# Dataset1. Create a line plot of Terature versus the predicted Ozone values
best_fit_line = air_ozone_predicted.hvplot.line(
    x = "TEMP",
    y = "Ozone_predicted",
    color = "orange"
)
best_fit_line

Dataset1. Plot of original Scatter Plot and Best Fit Line Plot

In [ ]:
                                 # Dataset1. Superposed scatter plot with original single independent variable:'TEMP'and dependent variable: O3 data, 
                                                           # and the best fit line model
air_plot * best_fit_line

Dataset1. Manual Predictions

In [21]:
# Dataset1. Display the formula to predict the sales with 100 ads
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]} * 100")

# Predict the sales with 100 ads
y_100 = model.intercept_ + model.coef_[0] * 100

# Display the prediction
print(f"Predicted Ozone (O3) metrics with Dataset1 {y_100:.2f}")

Model's formula: y = 17.31191562019925 + 2.963656055355665 * 100
Predicted Ozone (O3) metrics with Dataset1 313.68


Dataset1. Predictions Using the `Predict` Function

In [22]:
# Dataset1. Create an array to predict ozone levels for -0.5, -1.0, -1.5, -2.0, and -2.5 Temps
X_temps = np.array([-0.5, -1.0, -1.5, -2.0, -2.5])

# Format the array as a one-column array
X_temps = X_temps.reshape(-1,1)

# Display sample data
X_temps

array([[-0.5],
       [-1. ],
       [-1.5],
       [-2. ],
       [-2.5]])

In [23]:
# Dataset1. Predict ozone measures for independent single variable TEMP measures. 
predicted_ozone_temp = model.predict(X_temps)

In [24]:
# Dataset1. Create a DataFrame for the predicted ozone levels
df_predicted_ozone_temp = pd.DataFrame(
    {
        "temp": X_temps.reshape(1, -1)[0],
        "predicted_ozone": predicted_ozone_temp
    }
)

# Display data
df_predicted_ozone_temp

,temp,predicted_ozone
0,-0.5,15.830088
1,-1.0,14.348260
2,-1.5,12.866432
3,-2.0,11.384604
4,-2.5,9.902775


Data 1. Evaluating the Linear Regression Model 

In [25]:
# Dataset1. Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = model.score(X, y, sample_weight=None)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.3565257008834236.
The r2 is 0.3565257008834236.
The mean squared error is 2069.351378037146.
The root mean squared error is 45.490123961549564.
The standard deviation is 56.70893843043617.


Dataset 2 with Null values filled in with mean of their respective columns. 
Linear Regression Model process repeated using the cleansed dataset 2. 'data_med.csv'

In [26]:
# Dataset2. Read in cleansed data file 'data_drop.csv' from AWS S3 Bucket

url="https://project-4-group-6-air-quality.s3.us-east-2.amazonaws.com/data_med.csv"
air2_data2_df = pd.read_csv(
    url,
    sep=',',
    encoding='utf-8',
)
# Display sample data
air2_data2_df.head()

,Unnamed: 0,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM
0,0,2013,3,1,0,9.0,9.0,3.0,17.0,300.0,89.0000,-0.5,1024.5,-21.4,0.0,NNW,5.7
1,1,2013,3,1,1,4.0,4.0,3.0,16.0,300.0,88.0000,-0.7,1025.1,-22.1,0.0,NW,3.9
2,2,2013,3,1,2,7.0,7.0,10.0,17.0,300.0,60.0000,-1.2,1025.3,-24.6,0.0,NNW,5.3
3,3,2013,3,1,3,3.0,3.0,5.0,18.0,1000.0,44.1252,-1.4,1026.2,-25.5,0.0,N,4.9
4,4,2013,3,1,4,3.0,3.0,7.0,47.0,200.0,84.0000,-1.9,1027.1,-24.5,0.0,NNW,3.2


In [27]:
# Dataset2. Drop the 'wd' string object, and date columns,  and save to new df
air_data2_df = air2_data2_df.drop(columns=["Unnamed: 0",'year', 'month', 'day','hour','wd'])
air_data2_df

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
0,9.0,9.0,3.0,17.0,300.0,89.0000,-0.5,1024.5,-21.4,0.0,5.7
1,4.0,4.0,3.0,16.0,300.0,88.0000,-0.7,1025.1,-22.1,0.0,3.9
2,7.0,7.0,10.0,17.0,300.0,60.0000,-1.2,1025.3,-24.6,0.0,5.3
3,3.0,3.0,5.0,18.0,1000.0,44.1252,-1.4,1026.2,-25.5,0.0,4.9
4,3.0,3.0,7.0,47.0,200.0,84.0000,-1.9,1027.1,-24.5,0.0,3.2
...,...,...,...,...,...,...,...,...,...,...,...
420763,11.0,11.0,2.0,2.0,200.0,99.0000,11.7,1008.9,-13.3,0.0,1.3
420764,13.0,13.0,2.0,2.0,200.0,101.0000,10.9,1009.0,-14.0,0.0,2.1
420765,9.0,14.0,2.0,2.0,200.0,102.0000,9.5,1009.4,-13.0,0.0,1.5
420766,10.0,12.0,2.0,2.0,200.0,97.0000,7.8,1009.6,-12.6,0.0,1.4


Dataset2. Scatter Plot with data in the cleaned data_med.csv file for Ozone (O3) and Temperature (TEMP)

In [ ]:
# Dataset2. Create a scatter plot
air_plot = air_data2_df.hvplot.scatter(
    x="TEMP",
    y="O3",
    title="Dataset2: Expected Ozone Measures based on Temperature"
)
air_plot

In [28]:
# Dataset2. Create the X set by using the `reshape` function to format the TEMP data as a single column array.
X = air_data2_df["TEMP"].values.reshape(-1, 1)

# Display sample data
X[:5]

array([[-0.5],
       [-0.7],
       [-1.2],
       [-1.4],
       [-1.9]])

In [29]:
# Dataset2. Create an array for the dependent variable y with the O3 data
y = air_data2_df["O3"]

Dataset2.  Linear Regression Model with SciKit-learn

In [30]:
# Dataset2. Create a model with scikit-learn
model2 = LinearRegression()

In [31]:
# Dataset2 Fit the data into the model
model2.fit(X, y)

LinearRegression()

In [32]:
# Dataset2 Display the slope
print(f"Model's slope: {model2.coef_}")

Model's slope: [2.86612091]


In [33]:
# Dataset2. Display the y-intercept
print(f"Model's y-intercept: {model2.intercept_}")

Model's y-intercept: 18.118847608324636


In [34]:
# Dataset2. Display the model's best fit line formula
print(f"Model's formula: y = {model2.intercept_} + {model2.coef_[0]}X")

Model's formula: y = 18.118847608324636 + 2.866120914930229X


Data2.  Plot of Best Fit Line for the Prediction Model

In [35]:
# Dataset2. Make predictions using the X set
predicted_y_values = model2.predict(X)

In [36]:
# Dataset2. Create a copy of the original data
ozone_data2_predicted = air_data2_df.copy()

# Add a column with the predicted sales values
ozone_data2_predicted["Ozone_data2_predicted"] = predicted_y_values

# Display sample data
ozone_data2_predicted.head()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM,Ozone_data2_predicted
0,9.0,9.0,3.0,17.0,300.0,89.0000,-0.5,1024.5,-21.4,0.0,5.7,16.685787
1,4.0,4.0,3.0,16.0,300.0,88.0000,-0.7,1025.1,-22.1,0.0,3.9,16.112563
2,7.0,7.0,10.0,17.0,300.0,60.0000,-1.2,1025.3,-24.6,0.0,5.3,14.679503
3,3.0,3.0,5.0,18.0,1000.0,44.1252,-1.4,1026.2,-25.5,0.0,4.9,14.106278
4,3.0,3.0,7.0,47.0,200.0,84.0000,-1.9,1027.1,-24.5,0.0,3.2,12.673218


In [ ]:
# Dataset2. Create a line plot of ads versus the predicted sales values
best_fit_line = ozone_data2_predicted.hvplot.line(
    x = "TEMP",
    y = "Ozone_data2_predicted",
    color = "orange"
)
best_fit_line

Dataset2.  Plot of inital Scatter Plot and Best Fit Line Plot

In [ ]:
                                        # Dataset2: Superposed scatter plot with original single independent variable:'TEMP'and dependent variable: O3 data,
                                                                               # and the best fit line model
air_plot * best_fit_line

Data2.  Manual Predictions

In [37]:
# Dataset2. Display the formula to predict the sales with 100 ads
print(f"Model's formula: y = {model2.intercept_} + {model2.coef_[0]} * 100")

# Predict the sales with 100 ads
y_100 = model2.intercept_ + model2.coef_[0] * 100

# Display the prediction
print(f"Predicted Ozone (O3) metrics with data2 {y_100:.2f}")

Model's formula: y = 18.118847608324636 + 2.866120914930229 * 100
Predicted Ozone (O3) metrics with data2 304.73


Data2. Predictions Using the `Predict` Function

In [38]:
# Dataset2. Create an array to predict ozone levels for -0.5, -1.0, -1.5, -2.0, and -2.5 Temps
X_temps = np.array([-0.5, -1.0, -1.5, -2.0, -2.5])

# Format the array as a one-column array
X_temps = X_temps.reshape(-1,1)

# Display sample data
X_temps

array([[-0.5],
       [-1. ],
       [-1.5],
       [-2. ],
       [-2.5]])

In [39]:
# Dataset2. Predict ozone for temp values
predicted_ozone_data2 = model2.predict(X_temps)

In [40]:
# Dataset2. Create a DataFrame for the predicted ozone levels
df_predicted_ozone_data2 = pd.DataFrame(
    {
        "temp": X_temps.reshape(1, -1)[0],
        "predicted_ozone": predicted_ozone_temp
    }
)

# Display data
df_predicted_ozone_data2

,temp,predicted_ozone
0,-0.5,15.830088
1,-1.0,14.348260
2,-1.5,12.866432
3,-2.0,11.384604
4,-2.5,9.902775


Data 2. Assessing the Linear Regression Model

In [41]:
# Dataset2. Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = model2.score(X, y, sample_weight=None)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.34434693063035227.
The r2 is 0.34434693063035227.
The mean squared error is 2043.6992288229917.
The root mean squared error is 45.20729176607455.
The standard deviation is 55.83049022103166.


Conclusion
* Decided to go with the Dataset 1 with cleaned data_drop.csv 
* Dataset 2 with the Null filled in with Median values may add more bias to the results.